# All Policies

Analyze the performance of our Whittle and Adaptive Policies

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import sys
sys.path.append('/usr0/home/naveenr/projects/food_rescue_preferences')

In [190]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys
from copy import deepcopy
import secrets
from itertools import combinations
from rmab.simulator import create_transitions_from_prob
from rmab.compute_whittle import arm_compute_whittle_multi_state, arm_compute_whittle

In [246]:
n_arms = 7
budget = 3

probabilities = [np.random.random() for i in range(n_arms)]

In [232]:
def total_prob(action):
    action = np.array(action)
    if np.sum(action) == 0:
        return 0
    return 1/np.sum(action)

In [233]:
all_transitions,probs_by_partition = create_transitions_from_prob("food_rescue",43,max_transition_prob=0.25)

In [243]:
state = [np.random.randint(0,1) for i in range(n_arms)]

In [256]:
def get_transitions(global_action,seed):
    np.random.seed(seed)
    transitions = np.zeros((n_arms,2,2,2))
    transitions = all_transitions[np.random.choice(list(range(100)),n_arms)]

    selection_chance = np.array(probabilities)*total_prob(global_action)*np.array(global_action)
    transitions[:,1,1,1] = (1-selection_chance)*transitions[:,1,1,1] + selection_chance*1
    transitions[:,1,1,0] = 1-transitions[:,1,1,0]
    return transitions

In [265]:
seed = 44

In [266]:
start_action = np.zeros((n_arms))
action = np.zeros((n_arms)) 
start_action[np.random.choice(list(range(n_arms)),size=budget,replace=False)] = 1

while False in (action == start_action):
    print(action,start_action)
    state_WI = []

    transitions = get_transitions(start_action,seed)
    for i in range(n_arms):
        state_WI.append(arm_compute_whittle(transitions[i,:,:,1], state[i], 0.9, 0, eps=1e-3,reward_function='combined',lamb=0.5,match_prob=0.5,match_probability_list=[],get_v=False,num_arms=n_arms))
    print(state_WI)
    selected_idx = np.argsort(state_WI)[-budget:][::-1]
    action = deepcopy(start_action)
    start_action = np.zeros(n_arms, dtype=np.int8)
    start_action[selected_idx] = 1
    print("Best response {}".format(start_action))

[0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 1. 0. 1.]
[0.14162063598632812, 0.13208389282226562, 0.16546249389648438, 0.19693374633789062, 0.13875961303710938, 0.14638900756835938, 0.10728836059570312]
Best response [0 0 1 1 0 1 0]
[1. 0. 0. 0. 1. 0. 1.] [0 0 1 1 0 1 0]
[0.14066696166992188, 0.13208389282226562, 0.16832351684570312, 0.20074844360351562, 0.13875961303710938, 0.14734268188476562, 0.09584426879882812]
Best response [0 0 1 1 0 1 0]


In [267]:
max_whittle_sum = 0
max_action = []

for i in range(2**n_arms):
    binary_rep = bin(i)[2:]
    action = [int(j) for j in binary_rep.zfill(n_arms)]
    if np.sum(action) <= budget:
        transitions = get_transitions(action,seed)
        state_WI = []
        for j in range(n_arms):
            if action[j] == 1:
                state_WI.append(arm_compute_whittle(transitions[j,:,:,1], state[j], 0.9, 0, eps=1e-3,reward_function='combined',lamb=0.5,match_prob=0.5,match_probability_list=[],get_v=False,num_arms=n_arms))
        if np.sum(state_WI) > max_whittle_sum:
            max_action = action
            max_whittle_sum = np.sum(np.array(state_WI))
max_action

[0, 0, 1, 1, 0, 1, 0]